In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import re, unidecode

In [2]:
user = '' # TODO: Add username here
passwd = '' # TODO: Add password here
db_ip = '' # TODO: Add database IP here
db_name = 'team_103'

engine = sa.create_engine(f'mysql://{user}:{passwd}@{db_ip}/{db_name}', echo=True)

In [3]:
def pascalCase(inp):
    # Remove all accesnts from the string
    inp = unidecode.unidecode(inp)
    
    # Check if the string is already in PascalCase
    if inp.isalpha():
        return inp.lower().replace('_', '').replace('-', '')
        # return ''.join(inp[0].upper() + inp[1:]).replace('_', '').replace('-', '')

    # Remove symbols from the string
    # inp = re.split(r'[^\w\s]', '', inp)

    # Split the string into words
    words = re.split(r'[^a-zA-Z0-9]', inp)
    # print(words)

    # Capitalize the first letter of each word
    words = [word.lower() for word in words]

    # Join the words together
    pascal_case = ''.join(words)

    return pascal_case.replace('_', '').replace('-', '')

# Some sanity checks
pascalCase('esports'), pascalCase('e-sports'), pascalCase('éthers'), pascalCase('salary_id')

('esports', 'esports', 'ethers', 'salaryid')

In [4]:
def open_df_from_csv(file, usecols, index_cols, dtype):
    df = pd.read_csv(
        file, 
        usecols=usecols, 
        dtype=dtype
    )
    # Format index columns to prevent duplicates.
    for idx_col in index_cols:
        if df[idx_col].dtype == str or df[idx_col].dtype == object:
            df[idx_col] = df[idx_col].apply(pascalCase)

    if len(index_cols) > 0:
        df = df.drop_duplicates(
            subset=index_cols, 
            keep='first',
            
        )
        
    df = df.rename(
        mapper=pascalCase, 
        axis=1
    )

    if len(index_cols) > 0:
        df = df.set_index(
            [pascalCase(x) for x in index_cols]
        )

    return df

def export_table(df, table_name, engine):
    df.to_sql(
        table_name, 
        con=engine, 
        if_exists='append', 
        index=True
    )


## Open, clean, and export each `.csv` file.

### Courses

In [5]:
courses_df = open_df_from_csv(
    'courses.csv',
    usecols=['CRN', 'Year', 'Term', 'Subject', 'Number', 'Name', 'Description', 'Credit Hours'],
    index_cols=['CRN'],
    dtype={'CRN': int, 'Year': int, 'Term': str, 'Subject': str, 'Number': int, 'Name': str, 'Description': str, 'Credit Hours': str}
)
print(len(courses_df), courses_df.columns)

export_table(courses_df, 'Courses', engine)

11277 Index(['year', 'term', 'subject', 'number', 'name', 'description',
       'credithours'],
      dtype='object')
2024-04-06 01:08:56,313 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-06 01:08:56,314 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:56,324 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-06 01:08:56,324 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:56,327 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-06 01:08:56,328 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:56,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:08:56,340 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Courses`
2024-04-06 01:08:56,341 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:56,400 INFO sqlalchemy.engine.Engine INSERT INTO `Courses` (crn, year, term, subject, number, name, description, credithours) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
2024-04-06 01:08:56,400 INFO sqlalchemy.engine

### Companies

In [6]:
companies_df = open_df_from_csv(
    'job_postings/company_details/companies.csv',
    usecols=['company_id','name','description','company_size','state','country','city','zip_code','address','url'],
    index_cols=['company_id'],
    dtype={'company_id': str, 'name': str, 'description': str, 'company_size': float, 'state': str, 'country': str, 'city': str, 'zip_code': str, 'address': str, 'url': str}
)
print(len(companies_df), companies_df.columns)

export_table(companies_df, 'Companies', engine)

11361 Index(['name', 'description', 'companysize', 'state', 'country', 'city',
       'zipcode', 'address', 'url'],
      dtype='object')
2024-04-06 01:08:57,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:08:57,455 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Companies`
2024-04-06 01:08:57,456 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:57,496 INFO sqlalchemy.engine.Engine INSERT INTO `Companies` (companyid, name, description, companysize, state, country, city, zipcode, address, url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2024-04-06 01:08:57,496 INFO sqlalchemy.engine.Engine [generated in 0.02479s] [('1009', 'IBM', 'At IBM, we do more than work. We create. We create as technologists, developers, and engineers. We create with our partners. We create with our compe ... (138 characters truncated) ... e want to work with you.\n\nWe\'re here to help every creator turn their "what if" into what is. Let\'s create something that will change ever

### CompanyIndustries

In [7]:
company_industries_df = open_df_from_csv(
    'job_postings/company_details/company_industries.csv',
    usecols=['company_id','industry'],
    index_cols=['company_id','industry'],
    dtype={'company_id': str, 'industry': str}
)
print(len(company_industries_df), company_industries_df.index.is_unique)
company_industries_df = company_industries_df[company_industries_df.index.get_level_values('companyid').isin(companies_df.index.get_level_values('companyid').unique())]
print(len(company_industries_df), company_industries_df.columns)

export_table(company_industries_df, 'CompanyIndustries', engine)

12601 True
12556 Index([], dtype='object')
2024-04-06 01:08:59,597 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:08:59,598 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`CompanyIndustries`
2024-04-06 01:08:59,598 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:08:59,615 INFO sqlalchemy.engine.Engine INSERT INTO `CompanyIndustries` (companyid, industry) VALUES (%s, %s)
2024-04-06 01:08:59,615 INFO sqlalchemy.engine.Engine [generated in 0.00812s] [('91459053', 'internet'), ('69571610', 'cosmetics'), ('62424', 'realestate'), ('1387626', 'realestate'), ('2502541', 'informationtechnologyservices'), ('298409', 'staffingrecruiting'), ('37768', 'informationtechnologyservices'), ('7032083', 'informationtechnologyservices')  ... displaying 10 of 12556 total bound parameter sets ...  ('10135152', 'governmentadministration'), ('373873', 'itservicesanditconsulting')]
2024-04-06 01:08:59,852 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103`
2024-04-06 01:08:5

### CompanySpecialities

In [8]:
company_specialities_df = open_df_from_csv(
    'job_postings/company_details/company_specialities.csv',
    usecols=['company_id','speciality'],
    index_cols=['company_id','speciality'],
    dtype={'company_id': str,'speciality': str}
)
print(len(company_specialities_df), company_specialities_df.columns)
company_specialities_df = company_specialities_df[company_specialities_df.index.get_level_values('companyid').isin(companies_df.index.get_level_values('companyid').unique())]
company_specialities_df = company_specialities_df[company_specialities_df.index.get_level_values('speciality').str.len() <= 255]
print(len(company_specialities_df), company_specialities_df.columns)


export_table(company_specialities_df, 'CompanySpecialities', engine)

78268 Index([], dtype='object')
77930 Index([], dtype='object')
2024-04-06 01:09:00,069 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:00,073 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`CompanySpecialities`
2024-04-06 01:09:00,073 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:00,218 INFO sqlalchemy.engine.Engine INSERT INTO `CompanySpecialities` (companyid, speciality) VALUES (%s, %s)
2024-04-06 01:09:00,219 INFO sqlalchemy.engine.Engine [generated in 0.11637s] [('91459053', 'onlineprofessionalnetwork'), ('91459053', 'jobs'), ('91459053', 'peoplesearch'), ('91459053', 'companysearch'), ('91459053', 'addressbook'), ('91459053', 'advertising'), ('91459053', 'professionalidentity'), ('91459053', 'recruiting')  ... displaying 10 of 77930 total bound parameter sets ...  ('2293632', 'outdoordigitalsignage'), ('373873', 'systemintegratorforerpsandntierwebbasedprojects')]
2024-04-06 01:09:01,407 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103

### EmployeeCounts

In [9]:
employee_counts_df = open_df_from_csv(
    'job_postings/company_details/employee_counts.csv',
    usecols=['company_id','employee_count','follower_count','time_recorded'],
    index_cols=['company_id'],
    dtype={'company_id': str,'employee_count': int,'follower_count': int,'time_recorded': float}
)
print(len(employee_counts_df), employee_counts_df.columns)
employee_counts_df = employee_counts_df[employee_counts_df.index.get_level_values('companyid').isin(companies_df.index.get_level_values('companyid').unique())]
print(len(employee_counts_df), employee_counts_df.columns)

export_table(employee_counts_df, 'EmployeeCounts', engine)

11323 Index(['employeecount', 'followercount', 'timerecorded'], dtype='object')
11278 Index(['employeecount', 'followercount', 'timerecorded'], dtype='object')
2024-04-06 01:09:01,449 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:01,450 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`EmployeeCounts`
2024-04-06 01:09:01,450 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:01,467 INFO sqlalchemy.engine.Engine INSERT INTO `EmployeeCounts` (companyid, employeecount, followercount, timerecorded) VALUES (%s, %s, %s, %s)
2024-04-06 01:09:01,467 INFO sqlalchemy.engine.Engine [generated in 0.01031s] [('91459053', 2, 24, 1692644648.8276289), ('69571610', 3717, 158109, 1692644650.6863291), ('62424', 899, 10311, 1692644651.0513952), ('1387626', 1676, 39770, 1692644651.449499), ('2502541', 29, 8058, 1692644653.155199), ('298409', 395, 146046, 1692644653.5256968), ('37768', 2025, 231325, 1692644653.940611), ('7032083', 119, 40756, 1692644654.387137)  ... displaying 10

### JobPostings

In [10]:
job_postings_df = open_df_from_csv(
    "job_postings/job_postings.csv",
    usecols=[
        "job_id",
        "company_id",
        "title",
        "description",
        "formatted_work_type",
        "location",
        "applies",
        "original_listed_time",
        "remote_allowed",
        "views",
        "job_posting_url",
        "application_url",
        "application_type",
        "expiry",
        "closed_time",
        "formatted_experience_level",
        "skills_desc",
        "listed_time",
        "posting_domain",
        "sponsored",
        "work_type",
        "scraped",
    ],
    index_cols=["job_id"],
    dtype={"job_id": str,
        "company_id": str,
        "title": str,
        "description": str,
        "formatted_work_type": str,
        "location": str,
        "applies": float,
        "original_listed_time": float,
        "remote_allowed": 'Int64',
        "views": float,
        "job_posting_url": str,
        "application_url": str,
        "application_type": str,
        "closed_time": float,
        "formatted_experience_level": str,
        "skills_desc": str,
        "listed_time": float,
        "posting_domain": str,
        "sponsored": 'Int64',
        "work_type": str,
        "scraped": 'Int64',
    },
)
print(len(job_postings_df), job_postings_df.columns)
job_postings_df = job_postings_df[job_postings_df['companyid'].isin(companies_df.index.get_level_values('companyid').unique())]
print(len(job_postings_df), job_postings_df.columns)

export_table(job_postings_df, "JobPostings", engine)

33246 Index(['companyid', 'title', 'description', 'formattedworktype', 'location',
       'applies', 'originallistedtime', 'remoteallowed', 'views',
       'jobpostingurl', 'applicationurl', 'applicationtype', 'expiry',
       'closedtime', 'formattedexperiencelevel', 'skillsdesc', 'listedtime',
       'postingdomain', 'sponsored', 'worktype', 'scraped'],
      dtype='object')
32545 Index(['companyid', 'title', 'description', 'formattedworktype', 'location',
       'applies', 'originallistedtime', 'remoteallowed', 'views',
       'jobpostingurl', 'applicationurl', 'applicationtype', 'expiry',
       'closedtime', 'formattedexperiencelevel', 'skillsdesc', 'listedtime',
       'postingdomain', 'sponsored', 'worktype', 'scraped'],
      dtype='object')
2024-04-06 01:09:02,587 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:02,592 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`JobPostings`
2024-04-06 01:09:02,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:

### Benefits

In [11]:
benefits_df = open_df_from_csv(
    'job_postings/job_details/benefits.csv',
    usecols=['job_id','inferred','type',],
    index_cols=['job_id','type',],
    dtype={'job_id': str,'inferred': int,'type': str,}
)
print(len(benefits_df), benefits_df.columns)
benefits_df = benefits_df[benefits_df.index.get_level_values('jobid').isin(job_postings_df.index.get_level_values('jobid').unique())]
print(len(benefits_df), benefits_df.columns)

export_table(benefits_df, 'Benefits', engine)

29325 Index(['inferred'], dtype='object')
28621 Index(['inferred'], dtype='object')
2024-04-06 01:09:25,131 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:25,132 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Benefits`
2024-04-06 01:09:25,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:25,197 INFO sqlalchemy.engine.Engine INSERT INTO `Benefits` (jobid, type, inferred) VALUES (%s, %s, %s)
2024-04-06 01:09:25,197 INFO sqlalchemy.engine.Engine [generated in 0.05232s] [('3691799471', '401k', 1), ('3691799471', 'medicalinsurance', 1), ('3691799471', 'disabilityinsurance', 1), ('3692301169', 'medicalinsurance', 0), ('3692301169', 'visioninsurance', 0), ('3692301169', 'dentalinsurance', 0), ('3692303024', 'medicalinsurance', 0), ('3692303024', 'visioninsurance', 0)  ... displaying 10 of 28621 total bound parameter sets ...  ('3757934256', 'disabilityinsurance', 0), ('3757498232', 'medicalinsurance', 1)]
2024-04-06 01:09:25,988 INFO sqlalchemy.engine.Engine

### Salaries

In [12]:
salaries_df = open_df_from_csv(
    'job_postings/job_details/salaries.csv',
    usecols=['salary_id','job_id','max_salary','med_salary','min_salary','pay_period','currency','compensation_type',],
    index_cols=['salary_id'],
    dtype={'salary_id': str,'job_id': str,'max_salary': float,'med_salary': float,'min_salary': float,'pay_period':str,'currency': str,'compensation_type': str,}
)
print(len(salaries_df), salaries_df.columns)
salaries_df = salaries_df[salaries_df['jobid'].isin(job_postings_df.index.get_level_values('jobid').unique())]
print(len(salaries_df), salaries_df.columns)

export_table(salaries_df, 'Salaries', engine)

13352 Index(['jobid', 'maxsalary', 'medsalary', 'minsalary', 'payperiod', 'currency',
       'compensationtype'],
      dtype='object')
13116 Index(['jobid', 'maxsalary', 'medsalary', 'minsalary', 'payperiod', 'currency',
       'compensationtype'],
      dtype='object')
2024-04-06 01:09:26,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:26,045 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Salaries`
2024-04-06 01:09:26,046 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:26,079 INFO sqlalchemy.engine.Engine INSERT INTO `Salaries` (salaryid, jobid, maxsalary, medsalary, minsalary, payperiod, currency, compensationtype) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
2024-04-06 01:09:26,079 INFO sqlalchemy.engine.Engine [generated in 0.02079s] [('10', '3691799471', 135300.0, None, 98400.0, 'YEARLY', 'USD', 'BASE_SALARY'), ('11', '3691799474', 18.0, None, 16.0, 'HOURLY', 'USD', 'BASE_SALARY'), ('13', '3692301162', 73.0, None, 65.0, 'HOURLY', 'USD', 'BASE_SALARY

### Skills

In [13]:
skills_df = open_df_from_csv(
    'job_postings/maps/skills.csv',
    usecols=['skill_id','skill_name'],
    index_cols=['skill_id','skill_name'],
    dtype={'skill_id': str,'skill_name': str}
)
print(len(skills_df), skills_df.columns)

export_table(skills_df, 'Skills', engine)

35 Index([], dtype='object')
2024-04-06 01:09:26,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:26,881 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Skills`
2024-04-06 01:09:26,882 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:26,888 INFO sqlalchemy.engine.Engine INSERT INTO `Skills` (skillid, skillname) VALUES (%s, %s)
2024-04-06 01:09:26,889 INFO sqlalchemy.engine.Engine [generated in 0.00036s] [('prch', 'purchasing'), ('supl', 'supplychain'), ('pr', 'publicrelations'), ('sci', 'science'), ('stra', 'strategyplanning'), ('wrt', 'writingediting'), ('qa', 'qualityassurance'), ('dist', 'distribution')  ... displaying 10 of 35 total bound parameter sets ...  ('it', 'informationtechnology'), ('adm', 'administrative')]
2024-04-06 01:09:26,892 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103`
2024-04-06 01:09:26,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:26,896 INFO sqlalchemy.engine.Engine COMMIT


### Industries

In [14]:
industries_df = open_df_from_csv(
    'job_postings/maps/industries.csv',
    usecols=['industry_id','industry_name'],
    index_cols=['industry_id','industry_name'],
    dtype={'industry_id': int,'industry_name': str}
)
print(len(industries_df), industries_df.columns)

export_table(industries_df, 'Industries', engine)

212 Index([], dtype='object')
2024-04-06 01:09:26,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:26,912 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`Industries`
2024-04-06 01:09:26,912 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:26,917 INFO sqlalchemy.engine.Engine INSERT INTO `Industries` (industryid, industryname) VALUES (%s, %s)
2024-04-06 01:09:26,917 INFO sqlalchemy.engine.Engine [generated in 0.00035s] [(1, 'defenseandspacemanufacturing'), (3, 'computerhardwaremanufacturing'), (4, 'softwaredevelopment'), (5, 'computernetworkingproducts'), (6, 'technologyinformationandinternet'), (7, 'semiconductormanufacturing'), (8, 'telecommunications'), (9, 'lawpractice')  ... displaying 10 of 212 total bound parameter sets ...  (3243, 'servicesforrenewableenergy'), (3251, 'climatetechnologyproductmanufacturing')]
2024-04-06 01:09:26,921 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103`
2024-04-06 01:09:26,921 INFO sqlalchemy.engine.Engin

### JobSkills

In [15]:
job_skills_df = open_df_from_csv(
    'job_postings/job_details/job_skills.csv',
    usecols=['job_id','skill_id'],
    index_cols=['job_id','skill_id'],
    dtype={'job_id': str,'skill_id': str}
)
print(len(job_skills_df), job_skills_df.index.names)
job_skills_df = job_skills_df[job_skills_df.index.get_level_values('jobid').isin(job_postings_df.index.get_level_values('jobid').unique())]
print(len(job_skills_df), job_skills_df.columns)
job_skills_df = job_skills_df[job_skills_df.index.get_level_values('skillid').isin(skills_df.index.get_level_values('skillid').unique())]
print(len(job_skills_df), job_skills_df.columns)

export_table(job_skills_df, 'JobSkills', engine)

56591 ['jobid', 'skillid']
54760 Index([], dtype='object')
54760 Index([], dtype='object')
2024-04-06 01:09:27,045 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:27,047 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`JobSkills`
2024-04-06 01:09:27,047 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:27,159 INFO sqlalchemy.engine.Engine INSERT INTO `JobSkills` (jobid, skillid) VALUES (%s, %s)
2024-04-06 01:09:27,159 INFO sqlalchemy.engine.Engine [generated in 0.09615s] [('3691797924', 'eng'), ('3691797924', 'it'), ('3691799471', 'othr'), ('3691799471', 'rsch'), ('3691799474', 'mgmt'), ('3691799474', 'genb'), ('3691799474', 'adm'), ('3692300361', 'mrkt')  ... displaying 10 of 54760 total bound parameter sets ...  ('3757934256', 'it'), ('3757498232', 'adm')]
2024-04-06 01:09:28,640 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103`
2024-04-06 01:09:28,641 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:28,645 INFO sqlalchemy.engine.En

### JobIndustries

In [16]:
job_industries_df = open_df_from_csv(
    'job_postings/job_details/job_industries.csv',
    usecols=['job_id','industry_id'],
    index_cols=['job_id','industry_id'],
    dtype={'job_id': str,'industry_id': int}
)
print(len(job_industries_df), job_industries_df.columns)
job_industries_df = job_industries_df[job_industries_df.index.get_level_values('jobid').isin(job_postings_df.index.get_level_values('jobid').unique())]
print(len(job_industries_df), job_industries_df.columns)
job_industries_df = job_industries_df[job_industries_df.index.get_level_values('industryid').isin(industries_df.index.get_level_values('industryid').unique())]
print(len(job_industries_df), job_industries_df.columns)

export_table(job_industries_df, 'JobIndustries', engine)

44091 Index([], dtype='object')
42769 Index([], dtype='object')
42732 Index([], dtype='object')
2024-04-06 01:09:28,727 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:28,729 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`JobIndustries`
2024-04-06 01:09:28,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:28,770 INFO sqlalchemy.engine.Engine INSERT INTO `JobIndustries` (jobid, industryid) VALUES (%s, %s)
2024-04-06 01:09:28,771 INFO sqlalchemy.engine.Engine [generated in 0.02774s] [('3691797924', 6), ('3691799471', 25), ('3691799471', 18), ('3691799474', 44), ('3692300361', 44), ('3692301162', 96), ('3692301162', 14), ('3692301162', 4)  ... displaying 10 of 42732 total bound parameter sets ...  ('3757934256', 80), ('3757498232', 14)]
2024-04-06 01:09:29,807 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `team_103`
2024-04-06 01:09:29,808 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-06 01:09:29,831 INFO sqlalchemy.engine.Engine COMMIT


### CompanyReviews

In [17]:
# TODO: CompanyReviews, match CompanyIds
company_reviews_df = open_df_from_csv(
    "company_reviews.csv",
    usecols=[
        "name",
        "rating",
        "reviews",
        "description",
        "happiness",
        "ceo_approval",
        "ceo_count",
        "ratings",
        "locations",
        "roles",
        "salary",
        "interview_experience",
        "interview_difficulty",
        "interview_duration",
        "interview_count",
        "headquarters",
        "employees",
        "industry",
        "revenue",
        "website",
    ],
    index_cols=[],
    dtype={
        "name": str,
        "rating": float,
        "reviews": str,
        "description": str,
        "happiness": str,
        "ceo_approval": str,
        "ceo_count": str,
        "ratings": str,
        "locations": str,
        "roles": str,
        "salary": str,
        "interview_experience": str,
        "interview_difficulty": str,
        "interview_duration": str,
        "interview_count": str,
        "revenue": str,
        "website": str,
    },
)
print(len(company_reviews_df), company_reviews_df.columns)

# Inner join the two tables over company name.
company_reviews_df = company_reviews_df.merge(
    companies_df.reset_index(), on="name", how="inner", suffixes=(None, "_y")
)
company_reviews_df = company_reviews_df.drop_duplicates(
    subset=["name"],
    keep="first",
)
print(
    len(company_reviews_df),
    len(company_reviews_df["name"].unique()),
    company_reviews_df.columns,
)

company_reviews_df = company_reviews_df.drop(
    columns=[
        "name",
        "headquarters",
        "employees",
        "industry",
        "description_y",
        "companysize",
        "state",
        "country",
        "city",
        "zipcode",
        "address",
        "url",
    ]
)
print(len(company_reviews_df), len(company_reviews_df['companyid'].unique()))

company_reviews_df.index.names = ['reviewid']
print(company_reviews_df.index.names)

# company_reviews_df = company_reviews_df.set_index("companyid")


export_table(company_reviews_df, 'CompanyReviews', engine)

17050 Index(['name', 'rating', 'reviews', 'description', 'happiness', 'ceoapproval',
       'ceocount', 'ratings', 'locations', 'roles', 'salary',
       'interviewexperience', 'interviewdifficulty', 'interviewduration',
       'interviewcount', 'headquarters', 'employees', 'industry', 'revenue',
       'website'],
      dtype='object')
1372 1372 Index(['name', 'rating', 'reviews', 'description', 'happiness', 'ceoapproval',
       'ceocount', 'ratings', 'locations', 'roles', 'salary',
       'interviewexperience', 'interviewdifficulty', 'interviewduration',
       'interviewcount', 'headquarters', 'employees', 'industry', 'revenue',
       'website', 'companyid', 'description_y', 'companysize', 'state',
       'country', 'city', 'zipcode', 'address', 'url'],
      dtype='object')
1372 1372
['reviewid']
2024-04-06 01:09:30,019 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 01:09:30,020 INFO sqlalchemy.engine.Engine DESCRIBE `team_103`.`CompanyReviews`
2024-04-06 01:09:30,020 